In [1]:
%pwd

'/home/milad/projects/yoga-pose-detection/research'

In [2]:
import os
os.chdir('../')

In [3]:
%pwd

'/home/milad/projects/yoga-pose-detection'

In [64]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    resnet_base_model_path: Path
    resnet_updated_base_model_path: Path
    params_classes: int
    params_image_size: list
    params_pretrained: bool

In [65]:
from ypd.constants import *
from ypd.utils.common import read_yaml, create_directories

In [66]:
class configurationManager:
    def __init__(self, config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = config.root_dir,
            resnet_base_model_path = config.resnet_base_model_path,
            resnet_updated_base_model_path = config.resnet_updated_base_model_path,
            params_classes = self.params.CLASSES,
            params_image_size = self.params.IMAGE_SIZE,
            params_pretrained = self.params.PRETRAINED
        )
        
        return prepare_base_model_config

In [69]:
import os
from torchvision import models
import torch
from torchsummary import summary
from ypd import logger

In [15]:
a = [1,2,3,4,5]
a[:-3]

[1, 2]

In [36]:
resnet_model = models.resnet18(pretrained=True)

/home/milad/miniconda3/envs/kidney/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/milad/miniconda3/envs/kidney/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [63]:
resnet_model.fc.in_features


1000

In [ ]:
resnet_model.fc = torch.nn.Linear(resnet_model.fc.in_features, 5)

In [47]:
summary(resnet_model, (3, 224, 224))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [78]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    def get_base_model(self):
        self.resnet_model = models.resnet18(pretrained=self.config.params_pretrained)
        self.resnet_model.to(self.device)
        self.save_model(checkpoint=resnet_model, path=self.config.resnet_base_model_path)
        return resnet_model
    
    @staticmethod
    def _prepare_full_model(model, classes, freeze_till, freeze_all=False):
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        
        elif (freeze_till is not None) and (freeze_till > 0):
            for param in model.parameters()[:-freeze_till]:
                param.requires_grad = False
        
        n_inputs = model.fc.in_features
        model.fc = torch.nn.Linear(n_inputs, classes)
        return model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.get_base_model(),
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None
        )
        
        self.full_model.to(self.device)
        summary(self.full_model, input_size=tuple(self.config.params_image_size), device=self.device)
        self.save_model(checkpoint=self.full_model, path=self.config.resnet_updated_base_model_path)
        logger.info(f"saved updated model to {str(self.config.root_dir)}")

    
    @staticmethod
    def save_model(path: Path, checkpoint: dict):
        torch.save(checkpoint, path)
    

In [79]:
try:
    config = configurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.update_base_model()

except Exception as e:
    raise e
    

[2024-03-02 17:40:47,881: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-02 17:40:47,889: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-02 17:40:47,892: INFO: common: created directory at: artifacts]
[2024-03-02 17:40:47,893: INFO: common: created directory at: artifacts/prepare_base_model]


/home/milad/miniconda3/envs/kidney/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/milad/miniconda3/envs/kidney/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,